# Les implementations de trait

> Un trait définit une interface abstraite que les types peuvent implémenter. Cela permet aux types de partager des fonctionnalités communes sans avoir à partager une hiérarchie de classes commune. Nous avons précédemment croisé des traits utilises avec le mot-clef **derive** qui conféraient à nos structures ou a nos énumérations des capacités comme pouvoir debugger les valeurs en les affichant `Debug` ou bien un trait qui servait à la comparaison lors d'une condition `PartialEq`. Il existe aussi beaucoup d'autres traits qui ne sont pas dérivables et qui devraient être manuellement implémentés.
 
 ---

## Les traits derivables

### Le trait Debug

- Tous les types basiques que nous avons vu au chapitre 2 implémentent **Debug** via la std. Par exemple, si je cherche dans la documentation de la std des informations sur le type u64, dans la partie *Trait Implementations* à gauche, je peux y trouver Debug. Concrètement si je désire afficher la valeur de mon u64 via la macro `dbg!` ou via la macro `println!` avec en parametre `{:?}`, je peux visualiser cette valeur.

In [58]:
let j: u64 = 7;
dbg!(j);
println!("La valeur de j est {:?}", j);

[src/lib.rs:129] j = 7


Le type u64 implémente aussi le trait `Display`, trait qui permet d'afficher la valeur de façon "formatée", ce n'est plus un simple debug. Pour utiliser l'implémentation du trait Display de mon u64, je peux utiliser la macro println! avec en paramètre `{}`.

In [59]:
let j: u64 = 11;
println!("Valeur formatee de j: {}", j);

La valeur de j est 7
Valeur formatee de j: 11


*Cependant, Display n'est pas un trait dérivable contrairement à Debug, car si par exemple, je veux afficher une structure de façon formatée pour l'affichage final de mon programme, le compilateur ne peut deviner de quel façon je voudrais qu'il s'affiche !*

- Mais revenons au trait Debug, que se passe-t-il si je cherche à afficher les valeurs d'une structure Vector écrite pas mes soins ?

In [60]:
struct Vector {
    i: f64,
    j: f64,
}
let v1: Vector = Vector{
    i: 1.12,
    j: -2.1
};
dbg!(v1);

Error: `Vector` doesn't implement `Debug`

Error: `Vector` doesn't implement `Debug`

> Le trait `Debug` nécessite d'être implémenté pour notre structure. Debug est un trait qui permet de debugger, les types primitifs que l'on a vu jusqu'à maintenant implémentaient déjà Debug, mais ce n'est pas le cas de notre structure.
>
> Le compilateur nous propose d'utiliser un dérive pour cela, derive est une macro-procédurale de la std qui permet d'implémenter automatiquement certains traits. Utilisons donc ce derive sur la définition de notre type structure.
> 
> Notons enfin qu'**une structure ne peut dériver de Debug seulement si tous ses champs ont un type qui implémente Debug.**

In [61]:
#[derive(Debug)]
struct Vector {
    i: f64,
    j: f64,
}
let v1: Vector = Vector{
    i: 1.12,
    j: -2.1
};
dbg!(v1);

[src/lib.rs:132] v1 = Vector {
    i: 1.12,
    j: -2.1,
}


### Les traits Copy et Clone

- Tous les types basiques vus au chapitre 2 implémentent aussi les traits Copy et Clone, c'est-à-dire qu'il n'y a aucune restriction si je veux copier la valeur de ma variable a vers b, modifier celle de b puis pouvoir relire indépendamment a et b.

In [62]:
    let mut arr = [1, 3, 4];
    let arr2 = arr; // On copie les valeur de `arr` dans `arr2`
    arr[0] = 8; // On modifie le premier element de `arr`
    dbg!(arr);
    dbg!(arr2);

[src/lib.rs:132] arr = [
    8,
    3,
    4,
]
[src/lib.rs:134] arr2 = [
    1,
    3,
    4,
]


- Que se passe-t-il si je cherche à copier ma structure Vector de la même façon ?

In [63]:
#[derive(Debug)]
struct Vector {
    i: f64,
    j: f64,
}
let mut v1: Vector = Vector{
    i: 1.12,
    j: -2.1
};
let v2 = v1;
v1.i = -5.5;
dbg!(v1);
dbg!(v2);

Error: assign to part of moved value: `v1`

> Le trait `Copy` nécessite d'être implémenté pour notre structure ! Copy est un trait qui permet de copier la structure. Selon le compilateur la valeur de v1 a été 'move' dans v2 et n'est donc plus accessible lors de l'appel a dbg!. C'est le mécanisme d'ownership ou de possession que Rust a mis en place afin d'éviter des duplications mémoire de variables, si l'on veut qu'un type puisse être copie, il faudra obligatoirement implémenter Copy et Clone.
>
> Cela peut être aussi fait grâce à la directive dérive. Nous reviendrons plus tard sur Copy et Clone et ce sur ce mecanisme de move.
>
> Même règle que précédemment, **tous les champs doivent avoir un type qui soit Copy et Clone pour pouvoir deriver de Copy et de Clone**.

In [64]:
#[derive(Debug, Copy, Clone)]
struct Vector {
    i: f64,
    j: f64,
}
let mut v1 = Vector {
    i: 12.12,
    j: 13.32,
};
let v2 = v1;
v1.i = 3.14;
dbg!(v1);
dbg!(v2);

[src/lib.rs:132] v1 = Vector {
    i: 3.14,
    j: 13.32,
}
[src/lib.rs:133] v2 = Vector {
    i: 12.12,
    j: 13.32,
}


> **En résumé, pour l'instant, l'on fera précéder toutes nos structures par `#[derive(Debug, Copy, Clone)]`, nous reviendrons plus tard sur ces éléments.**

**Il existe d'autres traits dérivables:**
- *PartialEq* et *Eq* pour les comparaisons d'égalité
- *PartialOrd* et *Ord* pour les comparaisons
- *Hash* pour mapper une valeur à une valeur de taille fixe
- *Default* Valeur par défaut pour les valeurs par défaut

Enfin, on peut aussi se servir d'une crate comme *derive_more* afin de pouvoir encore plus utiliser de derives tels Add ou Sub.

- Exemple d'utilisation du trait Default :

In [65]:
#[derive(Copy, Clone, Debug, Default)]
struct Vector {
    i: f64,
    j: f64,
}
dbg!(Vector::default());

[src/lib.rs:128] Vector::default() = Vector {
    i: 0.0,
    j: 0.0,
}


## Des implementations que l'on ne peut deriver

### Le trait Drop

* Le trait Drop est invoqué quand une instance d'un type donné est détruit, que ce soit de la pile ou du tas. Implémenter le trait Drop pour un type sert généralement a exécuter quelque chose d'important avant sa destruction.

Si l'on implémente Drop pour une structure, ce n'est pas pour faire le boulot de nettoyage de la mémoire a la place du compilateur, mais seulement pour exécuter une action que l'on juge utile au moment de la destruction de la structure. On trouve quelques exemples dans des FFI bas niveau Rust / C. Mais Drop peut aussi aider parfois a débugger et à comprendre quand notre instance est supprimée de la mémoire, c'est utile pour les débutants qui testent Rust,

> Selon la documentation de std::ops::Drop...
> ```
pub trait Drop {
// Required method
fn drop(&mut self);
}
> ```
> ... pour implémenter le trait Drop, on doit définir une fonction nommée **drop** qui prend en paramètre une instance mutable de notre type note **self** en minuscule. Plus précisément une référence mutable de cette instance, nous ne verrons que plus tard ce qu'est une référence, ce n'est pas si important que ça pour l'instant.

* Exemple d'implémentation pour Vector

In [66]:
#[derive(Clone, Debug)]
struct Vector {
    i: f64,
    j: f64,
}
// Mon implementation de Drop pour Vector
impl std::ops::Drop for Vector {
    fn drop(&mut self) {
        println!("The vector is droped !");
    }
}
fn test() {
    let v: Vector = Vector {
        i: 3.13,
        j: 6.12,
    };
    dbg!(v);
    // A la fin du scope de ma fonction, le destructeur de la variable locale v est appele,
    // l'implementation du trait Drop() sera executee.
}
test();
println!("Fin de mon code");

[src/lib.rs:18] v = Vector {
    i: 3.13,
    j: 6.12,
}


### Additionner deux vecteurs

- Sans surprise, si maintenant, on cherche à additionner deux structures Vector, le compilateur refusera. Il ne sait pas comment additionner nos vecteurs et donc il ne se risque pas non plus a proposer un dérive qui additionnerait les premiers champs ensemble et les seconds champs ensemble. (*La crate derive_mode le fait ça !*)  

In [67]:
#[derive(Copy, Clone, Debug)]
struct Vector {
    i: f64,
    j: f64,
}
let v1 = Vector {
    i: 1.11,
    j: 3.33,
};
let v2 = v1; // Ici, ce sera une copie
let v3 = v1 + v2;

The vector is droped !
Fin de mon code


Error: cannot add `Vector` to `Vector`

* Hum.... Essayons un Derive...

In [68]:
#[derive(Add, Debug, Copy, Clone)]
struct Vector {
    i: f64,
    j: f64,
}

Error: cannot find derive macro `Add` in this scope

> **must implement `Add<_>`note: the trait `Add` must be implemented**

- Fonctionne pas ... On est donc dans l'obligation d'implementer le trait Add. Cela fera l'objet de l'exercice de fin de chapitre.

## Exercice

### En vous aidant de la documentation du trait `Add`, tentez d'implementer ce trait pour la structure Vector.

In [2]:
{
    #[derive(Debug, Copy, Clone)]
    struct Vector {
        i: f64,
        j: f64,
    }
    // Faire l'implementation ici
    let v3 = v1 + v2;
    dbg!(v3);
}

Error: cannot find value `v1` in this scope

Error: cannot find value `v2` in this scope

* Si on regarde la documentation du trait Add dans std::ops::Add:
>```
>pub trait Add<Rhs = Self> {
>    type Output;
>
>    // Required method
>    fn add(self, rhs: Rhs) -> Self::Output;
>}
>```
> - **Self** désigne le type de ce que l'on additionne. Self en majuscule désigne toujours un type.
> - **Output** doit être défini lors de l'implémentation. Bien que généralement, le type que l'on additionne donne en résultat le même type en sortie (Vector + Vector = Vector), il est possible que le type de sortie désire soit différent dans quelques cas particuliers (A + A = B).
> - **Rhs** veut dire Right Hand Side, c'est le terme de droite. Ici, le prototype du trait Add nous indique que l'on ne peut additionner que deux types identiques. (A + B) et jamais (A + B).
> - **self** désigne quant à lui une instance d'une variable de type Self, c'est le terme de gauche.

* En suivant l'exemple pour Point décrit dans la documentation. Il apparaît évident que ce sera quasiment la même chose que nous devrons écrire pour la structure Vector.

```
{
    #[derive(Debug, Copy, Clone)]
    struct Vector {
        i: f64,
        j: f64,
    }
    impl std::ops::Add for Vector {
        type Output = Self;

        fn add(self, other: Self) -> Self {
            Self {
                i: self.i + other.i,
                j: self.j + other.j,
            }
        }
    }
    let v3 = v1 + v2;
    dbg!(v3);
}
```